In [5]:
import os
import xarray as xr
import oggm
from oggm import utils,cfg, workflow,tasks
from oggm.workflow import execute_entity_task
import pandas as pd
import geopandas as gpd
import numpy as np
from time import gmtime, strftime
from cluster import  read_cmip6_data

In [22]:
REGION=11
JOB_NR=2000
GEOD_CALIB=True
cmip6_path='cmip6_gn'

cfg.initialize()
cfg.PATHS['working_dir'] = os.path.join('geod_run_CMIP6')
utils.mkdir(cfg.PATHS['working_dir'])

# Use multiprocessing?
cfg.PARAMS['use_multiprocessing'] = True

# store model geometry
cfg.PARAMS['store_model_geometry'] = True

# RGI file
path = utils.get_rgi_region_file(REGION, version='61')
rgidf = gpd.read_file(path)
rgidf = rgidf.sort_values('Area', ascending=True)

# exclude non-landterminating glaciers
rgidf = rgidf[rgidf.TermType == 0]
rgidf = rgidf[rgidf.Connect != 2]

if GEOD_CALIB:
    cfg.PARAMS['border'] = 240
    cfg.PARAMS['climate_qc_months']=0
    cfg.PARAMS['hydro_month_nh']=1
    cfg.PARAMS['hydro_month_sh']=1

    prepro_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/CRU/elev_bands/qc0/pcp2.5/match_geod_pergla/'
    url = os.path.join(prepro_url, 'RGI62/b_240/L5/summary/')
else:
    cfg.PARAMS['border'] = 160
    prepro_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/CRU/elev_bands/qc3/pcp2.5/no_match/'
    url = os.path.join(prepro_url, 'RGI62/b_160/L5/summary/')

# exculde glaciers that failed during preprocessing
fpath = utils.file_downloader(url + f'glacier_statistics_{REGION}.csv')
stat = pd.read_csv(fpath, index_col=0, low_memory=False)
rgidf = rgidf[~rgidf.RGIId.isin(stat.error_task.dropna().index)].reset_index()

#select glacier by JOB_NR
rgidf = rgidf.iloc[[JOB_NR]]

# Go - initialize glacier directories
gdirs = workflow.init_glacier_regions(rgidf, from_prepro_level=5, prepro_base_url=prepro_url)
gdir=gdirs[0]
# read (reset=False) or process cmip6 data (reset=True)
gcm_list = read_cmip6_data(cmip6_path, gdirs, reset=True)

2022-01-25 10:44:33: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2022-01-25 10:44:33: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2022-01-25 10:44:33: oggm.cfg: Multiprocessing: using all available processors (N=8)
2022-01-25 10:44:33: oggm.cfg: Multiprocessing switched ON after user settings.
2022-01-25 10:44:33: oggm.cfg: PARAMS['store_model_geometry'] changed from `False` to `True`.
2022-01-25 10:44:34: oggm.cfg: PARAMS['border'] changed from `40` to `240`.
2022-01-25 10:44:34: oggm.cfg: PARAMS['climate_qc_months'] changed from `3` to `0`.
2022-01-25 10:44:34: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2022-01-25 10:44:34: oggm.cfg: PARAMS['hydro_month_sh'] changed from `4` to `1`.
2022-01-25 10:44:34: oggm.workflow: init_glacier_regions from prepro level 5 on 1 glaciers.
2022-01-25 10:44:34: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 1 glaciers
2022-01-25 10:44:34: oggm.utils: No kn

In [23]:
for i, gcm in enumerate(gcm_list[1:]):
    print(gcm)
    if gcm != 'CRU':
        climate_filename='gcm_data'
        input_suffix=gcm
    else:
        climate_filename='climate_historical'
        input_suffix=None
    seed=1
    c = xr.open_dataset(gdir.get_filepath(climate_filename, filesuffix=input_suffix))
    years =  range(c.time.to_series().iloc[0].year + 16, c.time.to_series().iloc[-1].year - 14)
    yr=years[-1]
    mod = tasks.run_random_climate(gdir, climate_filename=climate_filename, climate_input_filesuffix=input_suffix, y0=yr,
                                             nyears=200, unique_samples=True, output_filesuffix=gcm + '_' + str(yr),
                                             seed=seed)

2022-01-25 10:44:40: oggm.core.flowline: (RGI60-11.02084) run_random_climateCMCC-CM2-HR4_1999
2022-01-25 10:44:40: oggm.core.flowline: (RGI60-11.02084) flowline_model_runCMCC-CM2-HR4_1999


CMCC-CM2-HR4


2022-01-25 10:44:47: oggm.core.flowline: (RGI60-11.02084) run_random_climateCESM2-FV2_1999
2022-01-25 10:44:47: oggm.core.flowline: (RGI60-11.02084) flowline_model_runCESM2-FV2_1999


CESM2-FV2


In [25]:
mod.volume_km3

0.008326372431561369